# Расчет цены на автомобиль

Постановка задачи:
Необходимо разработать модель, которая бы рассчитывала цену на выставленный на продажу автомобиль.  По условиям учебной задачи обучающих данных в явном виде не предоставлено. Только тестовые, собранные на авто-ру больше года назад. Необходимо самостоятельно разработать программу, которая бы собирала данные по объявлениям на том же сайте авто.ру.
Дополнительная сложность - количество данных. Оцениваться работа будет по порядка 35к записей. Необходимо собрать порядка 140 тыс записей.  На самом сайте автору сейчас актуально порядка 90к объявлений.


Решение задачи будет разделено на несколько этапов:
- EDA
- FE
- ...


In [1]:
import pandas as pd

## Знакомство с данными

На сайте kaggle имеются тестовый набор данных и набор данных от сентября 2020 года.  Попробуем проанализировать их.

In [2]:
train = pd.read_csv('input/all_auto_ru_09_09_2020.csv')
train.shape

(89378, 26)

In [3]:
test = pd.read_csv('input/test.csv')
test.shape

(34686, 32)

In [5]:
train.columns

Index(['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Таможня', 'Владение', 'price', 'start_date',
       'hidden', 'model'],
      dtype='object')

In [6]:
test.columns

Index(['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня'],
      dtype='object')

In [7]:
list(test.brand.unique())

['SKODA',
 'AUDI',
 'HONDA',
 'VOLVO',
 'BMW',
 'NISSAN',
 'INFINITI',
 'MERCEDES',
 'TOYOTA',
 'LEXUS',
 'VOLKSWAGEN',
 'MITSUBISHI']

In [8]:
list(train.brand.unique())

['AUDI',
 'BMW',
 'CADILLAC',
 'CHERY',
 'CHEVROLET',
 'CHRYSLER',
 'CITROEN',
 'DAEWOO',
 'DODGE',
 'FORD',
 'GEELY',
 'HONDA',
 'HYUNDAI',
 'INFINITI',
 'JAGUAR',
 'JEEP',
 'KIA',
 'LEXUS',
 'MAZDA',
 'MINI',
 'MITSUBISHI',
 'NISSAN',
 'OPEL',
 'PEUGEOT',
 'PORSCHE',
 'RENAULT',
 'SKODA',
 'SUBARU',
 'SUZUKI',
 'TOYOTA',
 'VOLKSWAGEN',
 'VOLVO',
 'GREAT_WALL',
 'LAND_ROVER',
 'MERCEDES',
 'SSANG_YONG']

Данные существенно отличаются по составу (названиям) столбцов и количеству брендов в наборах.  Выгружу данные для совместного исследования возможности объединения таблиц в Excel.

In [14]:
train.head(1).to_excel('train01.xls')
test.head(1).to_excel('test01.xls')
train.groupby(['brand','model']).price.count().to_excel('Train_Brand.xls')
test.groupby(['brand', 'model_name']).parsing_unixtime.count().to_excel('TestBrand.xls')

/var/folders/gb/vp2kjpjj7kxgkxk0ltq2p87h0000gn/T/ipykernel_81330/2826245866.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  train.head(1).to_excel('train01.xls')
/var/folders/gb/vp2kjpjj7kxgkxk0ltq2p87h0000gn/T/ipykernel_81330/2826245866.py:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is depre

### Предварительные выводы

Исследование структуры таблиц в Excel показало сложность приведения форматов двух таблиц к единому знаменателю. Кроме того, на вводной постановке задачи Ментор настоятельно рекомендовал собрать данные самостоятельно.  Попробую собрать данные с использованием решения, унаследованного от предыдущих студентов и найденого на kaggle/git_hub.

Для сбора данных с сайта выделена отдельная книга P05_02_AutoRu Parser.

При сборе данных возникнет еще одна проблема - временной характер цен, изменяющихся с течением времени. Необходимо где-то взять данные за прошлый период и сделать поправку на изменение цен. В ходе постановки задачи было предложено скорректировать цены, например на инфляцию.  Попробую вытащить средние цены по маркам из набора данных с сайта kaggle и в дальнейшем как-то учесть эту информацию.

In [12]:
train.groupby(['brand']).price.mean().to_csv('200909-BrandPrices.csv')
